In [ ]:
%matplotlib inline

# Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
url_format = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/{}/netcdf/A5B2/1.00/woa13_A5B2_{}{:02}_01v2.nc'

In [ ]:
temp_urls = [url_format.format('temperature', 't', i+1) for i in range(12)]
salt_urls = [url_format.format('salinity', 's', i+1) for i in range(12)]

Now we define a function that gives us a potential density section from a given month (default January), and range of latitudes and longitudes.

In [ ]:
def sect(lat, lon, month=0):
    salt_data = Dataset(salt_urls[month])
    temp_data = Dataset(temp_urls[month])
    
    depth = salt_data.variables['depth'][:]
    # calculate indices for lat/lon
    lat_d = salt_data.variables['lat'][:]
    lon_d = salt_data.variables['lon'][:]
    lat_i = (lat.min() <= lat_d) & (lat_d <= lat.max())
    lon_i = (lon.min() <= lon_d) & (lon_d <= lon.max())
    
    # compute absolute salinity from practical salinity
    sp = salt_data.variables['s_an'][0,:,lat_i,lon_i]
    print(sp.shape)
    sa = gsw.SA_from_SP(sp, depth, lon, lat)
    print(sa.shape, sa)
    
    # compute conservative temperature from in-situ temperature and absolute salinity
    ct = gsw.CT_from_t(sa, temp_data.variables['t_an'][0,:,lat_i,lon_i], depth)
    
    salt_data.close()
    temp_data.close()
    
    # compute potential density wrt surface
    return gsw.rho(sa, ct, 0)

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.

## Denmark Strait
Because it's pretty easy to find, we'll look at the Denmark strait from 22.5 to 39.5 degrees West, at 63.5 degrees North.

In [ ]:
temp = Dataset('data/woa13_A5B2_t01_01v2.nc', 'r')
salt = Dataset('data/woa13_A5B2_s01_01v2.nc', 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_i = (-39.5 <= lon) & (lon <= -22.5)

t_sect = temp.variables['t_an'][0,:,lat==63.5,lon_i].squeeze()
s_sect = salt.variables['s_an'][0,:,lat==63.5,lon_i].squeeze()
lon_sect = lon[lon_i]

Using TEOS-10, we can convert from practical salinity to absolute salinity, and from in-situ temperature to conservative temperature. From here, we can compute the locally-referenced density, and the potential density referenced to 2000m.

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, -39.5 + i, 63.5)
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

Let's take a look at the two density sections. The locally-refenced density shows very linear stratification (this is probably expected?), whereas the potential density shows a much clearer mixed layer (also as expected).

In [ ]:
ax = plt.subplot(121)
plt.pcolormesh(lon_sect, dep, rho_sect)
ax.invert_yaxis()
#plt.colorbar()

ax = plt.subplot(122)
plt.pcolormesh(lon_sect, dep, rhop_sect)
ax.invert_yaxis()
plt.colorbar()

We're looking at January, so this should be a Winter mixed layer. It's around 100m deep in the shallower regions, but nearly up to 400m deep in the deeper parts of the middle of the strait.

In [ ]:
for i in range(rhop_sect.shape[1]):
    plt.plot(rhop_sect[:,i], dep, '*')
plt.gca().invert_yaxis()
plt.grid(axis='y')

In [ ]:
# mixed layer depth for each column
mld = np.array([50, 50, 100, 200, 200, 200, 200, 200, 200, 200, 300, 300, 350, 350, 300, 100, 100, 100])

To give some information between water columns, we want to compute neutral density differences. We can use the thermal expansion and haline contraction coefficients from the mean salinity, temperature and pressure of the water parcels, as well as the local density from the mean.

In [ ]:
rhop_sect[30,5:7]

In [ ]:
sa_c = (sa_sect[30,5] + sa_sect[30,6]) / 2
ct_c = (ct_sect[30,5] + ct_sect[30,6]) / 2

In [ ]:
rho_c, alpha_c, beta_c = gsw.rho_alpha_beta(sa_c, ct_c, dep[30])

In [ ]:
nd_diff = rho_c * (beta_c * (sa_sect[30,5] - sa_sect[30,6]) - alpha_c * (ct_sect[30,5] - ct_sect[30,6]))
print(nd_diff)

Now we want to compute a distance (in physical space) by which to shift the parcel in order to flatten out the difference.

In [ ]:
g = 9.7963 # value of gravity used in gsw

In [ ]:
n2 = gsw.Nsquared(sa_sect[29:31,5],
                  ct_sect[29:31,5],
                  dep[29:31])
dz = (g**2 * nd_diff) / (1e4 * n2[0])
print(dz)

# Grid Generation

We can take our initial grid (which is set by the depths of observations, some of which are masked) and convert that to a representative model grid with millimetre-thick layers instead of completely missing layers.

In [ ]:
# depths of values with local mask
gr_dep = np.ma.array(np.tile(dep.reshape(-1, 1), (1, t_sect.shape[1])), mask=t_sect.mask)

# deepest data within each column
topo = gr_dep.max(axis=0).compressed().reshape(1,-1)

# interface positions between valid data
gr_int = (gr_dep[1:] + gr_dep[:-1]) / 2
m = gr_int.mask
gr_int = gr_int.filled()

# fill masks within each column with topo
# add a millimetre for each successive masked value
# so we don't have anything with exactly zero thickness
gr_int[m] = (np.tile(topo, (gr_int.shape[0], 1)) + 1e-3 * np.cumsum(m, axis=0))[m]

# prepend 0 to each column, append topo
gr_int = np.concatenate((np.zeros((1, gr_int.shape[1])), gr_int, topo), axis=0)
gr_lay = (gr_int[1:,:] + gr_int[:-1,:]) / 2

# calculate thickness
gr_h   = np.diff(gr_int, axis=0)

We can convert our temperature and salinity data from masked arrays to being defined on a full-depth grid of a generalised vertical coordinate, where masked values become vanished layers.

In [ ]:
m = sa_sect.mask
bot_i = np.cumsum(~m, axis=0).max(axis=0) - 1

# absolute salinity for remapping
sa_bot = sa_sect[bot_i,np.arange(sa_sect.shape[1])].compressed().reshape(1,-1)
sa_map = sa_sect.filled() # un-mask array
sa_map[m] = np.tile(sa_bot, (sa_sect.shape[0], 1))[m]

# conservative temperature
ct_bot = ct_sect[bot_i,np.arange(ct_sect.shape[1])].compressed().reshape(1,-1)
ct_map = ct_sect.filled()
ct_map[m] = np.tile(ct_bot, (ct_sect.shape[0], 1))[m]

We can go about actually generating the grid from the data in a couple of different ways:

The first way is to use the Burchard and Beckers (2004) vertical grid diffusion with a coefficient representing the mixed layer region, and another representing the adjustment required to minimise local neutral density curvature.

The second way is more along the lines of Hofmeister et al. (2010). This still involves a vertical grid diffusion, but we're not using any of the terms in this equation, so perhaps we should ignore it. Then we're left with the isopycnal (or in our case, neutral density difference) tendency term. Because this may lead to invalid layers, we follow up with a corrective step that's weighted in order to preserve resolution in the mixed layer.

## Neutral Density Difference

Using ghost cells, we can calculate the neutral density difference for every cell. Not sure what to do about vanished cells, but for the moment we'll just give them the same value as the first non-vanished cell above them in the same column.

In [ ]:
def ndd(z_lay, sa_lay, ct_lay):
    """
    Calculate neutral density difference (curvature) between
    adjacent columns, given their (absolute) salinity, (conservative)
    temperature and physical positions (or pressure).
    """
    
    # for dealing with edges, extend data with a ghost column
    # layer positions with ghosts
    #"""
    lay_gst = np.concatenate([ z_lay[:,[1]],  z_lay,  z_lay[:,[-2]]], axis=1)
    sa_gst  = np.concatenate([sa_lay[:,[1]], sa_lay, sa_lay[:,[-2]]], axis=1)
    ct_gst  = np.concatenate([ct_lay[:,[1]], ct_lay, ct_lay[:,[-2]]], axis=1)
    """
    lay_gst = np.concatenate([ z_lay[:,[0]],  z_lay,  z_lay[:,[0]]], axis=1)
    sa_gst  = np.concatenate([sa_lay[:,[0]], sa_lay, sa_lay[:,[0]]], axis=1)
    ct_gst  = np.concatenate([ct_lay[:,[0]], ct_lay, ct_lay[:,[0]]], axis=1)
    """
    
    # neutral density curvature *at layers*
    ndd = np.empty_like(z_lay)

    # calculate for each (real) column
    for i in range(sa_lay.shape[1]):
        # calculate difference to column at left
        sa_c =  sa_gst[:,[i,i+1]].mean(axis=1)
        ct_c =  ct_gst[:,[i,i+1]].mean(axis=1)
        z_c  = lay_gst[:,[i,i+1]].mean(axis=1)

        r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
        ndd_l = r * (b * (sa_gst[:,i+1] - sa_gst[:,i]) - a * (ct_gst[:,i+1] - ct_gst[:,i]))

        # calculate difference to column at right
        sa_c =  sa_gst[:,[i+2,i+1]].mean(axis=1)
        ct_c =  ct_gst[:,[i+2,i+1]].mean(axis=1)
        z_c  = lay_gst[:,[i+2,i+1]].mean(axis=1)

        r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
        ndd_r = r * (b * (sa_gst[:,i+2] - sa_gst[:,i+1]) - a * (ct_gst[:,i+2] - ct_gst[:,i+1]))

        ndd[:,i] = ndd_r - ndd_l
        
    return ndd

In [ ]:
def ndd_int(z_int, sa_lay, ct_lay):
    """
    Calculate neutral density difference (curvature) between
    adjacent columns, given their (absolute) salinity, (conservative)
    temperature and physical positions (or pressure).
    """
    
    sa_int = (sa_lay[1:,:] + sa_lay[:-1,:]) / 2
    ct_int = (ct_lay[1:,:] + ct_lay[:-1,:]) / 2
    
    # for dealing with edges, extend data with a ghost column
    # layer positions with ghosts
    int_gst = np.concatenate([ z_int[:,[1]],  z_int,  z_int[:,[-2]]], axis=1)
    sa_gst  = np.concatenate([sa_int[:,[1]], sa_int, sa_int[:,[-2]]], axis=1)
    ct_gst  = np.concatenate([ct_int[:,[1]], ct_int, ct_int[:,[-2]]], axis=1)
    
    # neutral density curvature *at layers*
    ndd = np.empty_like(z_int)

    # calculate for each (real) column
    for i in range(sa_int.shape[1]):
        # calculate difference to column at left
        sa_c =  sa_gst[:,[i,i+1]].mean(axis=1)
        ct_c =  ct_gst[:,[i,i+1]].mean(axis=1)
        z_c  = int_gst[:,[i,i+1]].mean(axis=1)

        r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
        ndd_l = r * (b * (sa_gst[:,i+1] - sa_gst[:,i]) - a * (ct_gst[:,i+1] - ct_gst[:,i]))

        # calculate difference to column at right
        sa_c =  sa_gst[:,[i+2,i+1]].mean(axis=1)
        ct_c =  ct_gst[:,[i+2,i+1]].mean(axis=1)
        z_c  = int_gst[:,[i+2,i+1]].mean(axis=1)

        r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
        ndd_r = r * (b * (sa_gst[:,i+2] - sa_gst[:,i+1]) - a * (ct_gst[:,i+2] - ct_gst[:,i+1]))

        ndd[:,i] = ndd_r - ndd_l
        
    return ndd

In [ ]:
def k_ndd_int(ndd_int):
    """
    Calculate k_ndd on layers depending on values of
    ndd at the surrounding interfaces.
    If ndd is diverging at a layer, there is a finite value of k_ndd
    otherwise, for converging ndd the diffusivity is zero.
    """
    
    # set ndd to zero at top and bottom boundaries
    z = np.zeros((1, ndd_int.shape[1]))
    ndd = np.concatenate((z, ndd_int, z), axis=0)
    
    # diverging -> positive k_ndd
    # converging -> zero k_ndd
    return np.maximum(ndd[1:,:] - ndd[:-1,:], 0)

In [ ]:
# if we calcluated ndd on interfaces,
# interpolate from interfaces to layers so we can define a diffusivity
#ndd_lay = np.empty((ndd.shape[0] + 1, ndd.shape[1]))
#for i in range(ndd.shape[1]):
#    f            = interp1d(gr_int[1:-1,i], ndd[:,i], bounds_error=False, fill_value="extrapolate")
#    ndd_lay[:,i] = f(gr_lay[:,i])

# otherwise, ndd is already in the right place
ndd_lay = ndd(gr_lay, sa_map, ct_map)
    
# calculate ndd diffusivity coefficients by normalising by top-to-bottom potential density difference
k_ndd = ndd_lay / (rhop_sect[bot_i,np.arange(rhop_sect.shape[1])] - rhop_sect[0,:])
#k_ndd = np.maximum(k_ndd, 0)
k_ndd = k_ndd**2

Let's take a look at these neutral density differences `ndd`, and also normalise them by the top-to-bottom potential density difference from `rhop_sect`.

In [ ]:
ax = plt.subplot(211)
plt.pcolormesh(lon_sect, gr_lay, ndd_lay)
ax.invert_yaxis()
plt.colorbar()

# normalise by potential density difference in each column
ax = plt.subplot(212)
plt.pcolormesh(lon_sect, gr_lay, k_ndd)
plt.colorbar()
ax.invert_yaxis()

## Diffusing interfaces

Now we can apply our diffusivity equation to interfaces, using the normalised `ndd` above as the grid diffusion coefficients. We start with interfaces at `gr_int`.

In [ ]:
def diffuse(z_int, k_ndd, c_ndd, c_surf, d_surf, dt):
    """
    Use an implicit diffusivity equation to evolve the grid defined by z_int at timestep dt.
    Neutral density difference diffusivity coefficient k_ndd is applied with weight c_ndd
    (background diffusivity is then applied with weight (1 - c_ndd)).
    """
    
    # allocate new grid
    z_next = np.empty_like(z_int)

    # iterate over columns
    for i in range(z_int.shape[1]):
        # determine total grid coefficient from
        # ndd term k_ndd
        # background term 1/D, where D is local depth
        # k_grid = D/tgrid * (c_ndd * k_ndd + c_b * k_b)
        k_grid = (z_int[-1,i] * (c_ndd * k_ndd[:,i] + c_surf / (d_surf + z_lay[:,i])) \
                  + (1 - c_ndd - c_surf)) / 3600

        I = z_int.shape[0] - 1 # number of layers
        A = np.zeros((3, I+1)) # diffusion system coefficients

        A[0,2:]     = -dt * I**2 * k_grid[1:]
        A[2,:-2]    = -dt * I**2 * k_grid[:-1]
        A[1,[0,-1]] = 1 # boundary conditions
        A[1,1:-1]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])

        # solve tridiagonal system
        z_next[:,i] = solve_banded((1, 1), A, z_int[:,i])
        
    return z_next

In [ ]:
gr_next = diffuse(gr_int, k_ndd, 0.3, 0.2, 50, 10)
h_next = np.diff(gr_next, axis=0)

In [ ]:
ax = plt.subplot(121)
plt.plot(gr_int)
ax.invert_yaxis()

ax = plt.subplot(122)
plt.plot(gr_next)
ax.invert_yaxis()

In [ ]:
ax = plt.subplot(211)
plt.pcolormesh(gr_h)
plt.colorbar()
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(h_next)
plt.colorbar()
ax.invert_yaxis()

## Remapping

Now that we've obtained new interface positions `gr_next`, we can remap and see if we've improved the local neutral density curvature!

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

In [ ]:
def remap(h):
    """
    Remap from original climatological grid according to h
    """

    sa_remap = np.empty_like(sa_map)
    ct_remap = np.empty_like(ct_map)

    for i in range(sa_remap.shape[1]):
        sa_remap[:,i] = mom_remapping.remapping_core_h(gr_h[:,i], sa_map[:,i], h[:,i], remap_cs)
        ct_remap[:,i] = mom_remapping.remapping_core_h(gr_h[:,i], ct_map[:,i], h[:,i], remap_cs)
        
    return sa_remap, ct_remap

In [ ]:
sa_remap, ct_remap = remap(h_next)

In [ ]:
ax = plt.subplot(221)
plt.pcolormesh(sa_map)
ax.invert_yaxis()

ax = plt.subplot(222)
plt.pcolormesh(sa_remap)
ax.invert_yaxis()

ax = plt.subplot(223)
plt.pcolormesh(ct_map)
ax.invert_yaxis()

ax = plt.subplot(224)
plt.pcolormesh(ct_remap)
ax.invert_yaxis()

## Iterative refinement

Now we can just encapsulate the whole process in a loop over timesteps. Remapping is always done from the source data, not progressively, so we can try to preserve the original structure.

In [ ]:
z_lay = gr_lay
z_int = gr_int
sa    = sa_map
ct    = ct_map

ndd_init = ndd(z_lay, sa, ct)
rhop_init = np.empty_like(sa)
for i in range(sa.shape[1]):
    rhop_init[:,i] = gsw.rho(sa[:,i], ct[:,i], 2000)

for k in range(10):
    # use 0.5 as reference density difference
    #k_ndd  = np.abs(ndd(z_lay, sa, ct) / 0.5)
    k_ndd  = k_ndd_int(ndd_int(z_int[1:-1], sa, ct)) / 0.5
    z_int  = diffuse(z_int, k_ndd, c_ndd=0.2, c_surf=0.4, d_surf=100, dt=10)
    z_lay  = (z_int[1:,:] + z_int[:-1,:]) / 2
    h      = np.diff(z_int, axis=0)
    sa, ct = remap(h)

First let's see the final `ndd` state, and how it has changed from the initial state. It looks here as though we're actually increasing the neutral density curvature...

In [ ]:
ndd_final = ndd(z_lay, sa, ct)

ax = plt.subplot(211)
plt.pcolormesh(ndd_final)
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(ndd_final - ndd_init)
ax.invert_yaxis()

Now let's look at layer thicknesses. Because `k_ndd` is quite discontinuous, depending on convergences and divergences, there are large changes in thickness within a column and within a layer.

In [ ]:
plt.pcolormesh(h)
plt.colorbar()
plt.gca().invert_yaxis()

In [ ]:
rhop_final = np.empty_like(sa)
for i in range(sa.shape[1]):
    rhop_final[:,i] = gsw.rho(sa[:,i], ct[:,i], 2000)

ax = plt.subplot(211)
plt.pcolormesh(rhop_sect)
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(lon_sect, z_lay, rhop_final)
ax.invert_yaxis()

In [ ]:
ax = plt.subplot(211)
plt.pcolormesh(rhop_final)
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(rhop_final - rhop_init)
ax.invert_yaxis()

In [ ]:
plt.plot(z_lay, '*')
plt.gca().invert_yaxis()

In [ ]:
plt.plot(z_lay - gr_lay, '*');